In [138]:
import os
import sys

import evo_helpers as hlp
import evo_constants as CTS

In [139]:
hlp.welcome()

Welcome to EvoMinig

Press enter to continune ...


In [ ]:
from importlib import reload
reload(evo_helpers)

opt = hlp.option_menu()

Press 1 to continue with precharged example.
Press 2 to continue with your own databases.


In [136]:
## obtener las bases de datos del usuario

from importlib import reload
reload(evo_helpers)

if opt == 1:
    evomining = EvoMining(cts.EXML_DB["central"],
                          cts.EXMPL_DB["genomes"],
                          cts.EXMPL_DB["nat_prods"])
    evomining.run()
else: 
    user_dbs = hlp.get_dbs()



 Enter path to central_DB: /data/central/

 Enter path to genomes_DB: /data/genomes/

 Enter path to rast_ids: /data/rast.ids

 Enter path to nat_prods_DB: data/mibig/


In [ ]:
##Obtener los headers que utiliza evomining:
